In [1]:
from qat.lang.AQASM import Program, H, CNOT

# Create a Program
qprog = Program()
# Number of qbits
nbqbits = 2
# Allocate some qbits
qbits = qprog.qalloc(nbqbits)

# Apply some quantum Gates
H(qbits[0])
CNOT(qbits[0], qbits[1])

# Export this program into a quantum circuit
circuit = qprog.to_circ()

# Import a Quantum Processor Unit Factory (the default one)
from qat.qpus import get_default_qpu

# Create a Quantum Processor Unit
qpu = get_default_qpu()

# Create a job
job = circuit.to_job()

# Submit the job to the QPU
result = qpu.submit(job)

# Iterate over the final state vector to get all final components
for sample in result:
    print("State %s amplitude %s" % (sample.state, sample.amplitude))

State |00> amplitude (0.7071067811865475+0j)
State |11> amplitude (0.7071067811865475+0j)


In [3]:
qbits?

Type:        QRegister
String form: QReg(q[0]..q[1])
Length:      2
File:        ~/miniconda3/envs/qc/lib/python3.10/site-packages/qat/lang/AQASM/bits.so
Docstring:  
Class for registers of qbits. They should only be
declared through the qalloc method of the Program class.
Qbits inside a register can be accessed
similarly to elements of an array (i.e using brackets and slices).
